<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/bondforward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest

class BondForwardTests(unittest.TestCase):

    class CommonVars:
        def __init__(self):
            # common data
            self.today = ql.Date(7, ql.March, 2022)
            ql.Settings.instance().evaluationDate = self.today

            self.curveHandle = ql.RelinkableYieldTermStructureHandle()
            self.curveHandle.linkTo(ql.FlatForward(self.today, 0.0004977, ql.Actual365Fixed()))

    def buildBond(self, issue_date: ql.Date, maturity_date: ql.Date, cpn: ql.Rate) -> ql.FixedRateBond:
        schedule = ql.Schedule(issue_date, maturity_date, ql.Period(ql.Annual), ql.TARGET(),
                               ql.Following, ql.Following,
                               ql.DateGeneration.Backward, False)

        # FixedRateBond(settlementDays, faceAmount, schedule, coupons, paymentDayCounter, ...)
        return ql.FixedRateBond(2,  # settlementDays
                               1.e5, # faceAmount
                               schedule,
                               [cpn], # coupons
                               ql.ActualActual(ql.ActualActual.ISDA)) # paymentDayCounter

    def buildBondForward(self,
                         underlying_bond: ql.Bond,
                         discount_curve_handle: ql.YieldTermStructureHandle,
                         delivery_date: ql.Date,
                         pos_type: ql.Position.Type) -> ql.BondForward:

        value_date = discount_curve_handle.referenceDate()

        # BondForward(valueDate, maturityDate (deliveryDate), type, strike, settlementDays,
        #             dayCounter, calendar, convention, underlyingBond, discountCurve, incomeDiscountCurve)
        return ql.BondForward(value_date,
                              delivery_date,
                              pos_type,
                              0.0,  # strike
                              2,    # settlementDays for the forward contract
                              ql.ActualActual(ql.ActualActual.ISDA), # dayCounter
                              ql.TARGET(), # calendar
                              ql.Following, # convention
                              underlying_bond,
                              discount_curve_handle, # discountCurve for forward
                              discount_curve_handle) # incomeDiscountCurve for forward

    def testFuturesPriceReplication(self):
        print("Testing futures price replication...")

        vars_ = self.CommonVars()
        tolerance = 1.0e-2

        issue = ql.Date(15, ql.August, 2015)
        maturity = ql.Date(15, ql.August, 2046)
        cpn_rate = 0.025

        bond = self.buildBond(issue, maturity, cpn_rate)
        pricer = ql.DiscountingBondEngine(vars_.curveHandle)
        bond.setPricingEngine(pricer)

        delivery = ql.Date(10, ql.March, 2022)
        conversion_factor = 0.76871

        bond_fwd = self.buildBondForward(bond, vars_.curveHandle, delivery, ql.Position.Long)

        futures_price = bond_fwd.cleanForwardPrice() / conversion_factor
        expected_futures_price = 207.47

        self.assertAlmostEqual(futures_price, expected_futures_price, delta=tolerance,
                               msg=(f"unable to replicate bond futures price\n"
                                    f"    calculated:    {futures_price:.5f}\n"
                                    f"    expected:    {expected_futures_price:.5f}\n"))

    def testCleanForwardPriceReplication(self):
        print("Testing clean forward price replication...")

        vars_ = self.CommonVars()
        tolerance = 1.0e-2

        issue = ql.Date(15, ql.August, 2015)
        maturity = ql.Date(15, ql.August, 2046)
        cpn_rate = 0.025

        bond = self.buildBond(issue, maturity, cpn_rate)
        pricer = ql.DiscountingBondEngine(vars_.curveHandle)
        bond.setPricingEngine(pricer)

        delivery = ql.Date(10, ql.March, 2022)
        bond_fwd = self.buildBondForward(bond, vars_.curveHandle, delivery, ql.Position.Long)

        fwd_clean_price = bond_fwd.cleanForwardPrice()
        # forwardValue() is the dirty forward price at delivery
        # accruedAmount(delivery) is the accrued interest on the bond at delivery
        expected_fwd_clean_price = bond_fwd.forwardValue() - bond.accruedAmount(delivery)

        self.assertAlmostEqual(fwd_clean_price, expected_fwd_clean_price, delta=tolerance,
                               msg=(f"unable to replicate clean forward price\n"
                                    f"    calculated:    {fwd_clean_price:.5f}\n"
                                    f"    expected:    {expected_fwd_clean_price:.5f}\n"))

    def testThatForwardValueIsEqualToSpotValueIfNoIncome(self):
        print("Testing that forward value is equal to spot value if no income (over short period with low rates)...")

        vars_ = self.CommonVars()
        tolerance = 1.0e-2

        issue = ql.Date(15, ql.August, 2015)
        maturity = ql.Date(15, ql.August, 2046) # Long-dated bond, annual coupons on Aug 15
        cpn_rate = 0.025

        bond = self.buildBond(issue, maturity, cpn_rate)
        pricer = ql.DiscountingBondEngine(vars_.curveHandle)
        bond.setPricingEngine(pricer)

        # Delivery date is very close to today (valueDate for the forward)
        # today = 7 March 2022. Bond settlement = 9 March 2022.
        # delivery = 10 March 2022.
        # No discrete coupons fall between bond settlement and delivery.
        delivery = ql.Date(10, ql.March, 2022)
        bond_fwd = self.buildBondForward(bond, vars_.curveHandle, delivery, ql.Position.Long)

        # bndFwd.forwardValue() is the expected dirty price of the bond at the delivery date
        bnd_fwd_value = bond_fwd.forwardValue()

        # bnd.dirtyPrice() is the dirty price of the bond for spot settlement (today + 2 days)
        underlying_dirty_price = bond.dirtyPrice()

        # For a very short forward period (1 day between spot settlement and fwd delivery here)
        # with no discrete cashflows and very low interest rates,
        # DirtyForwardPrice(at T_delivery) approx= DirtySpotPrice(at T_spot_settle)
        self.assertAlmostEqual(bnd_fwd_value, underlying_dirty_price, delta=tolerance,
                               msg=(f"unable to match the dirty price \n"
                                    f"    bond forward value (dirty price at delivery):    {bnd_fwd_value:.5f}\n"
                                    f"    underlying bond dirty price (spot settlement):    {underlying_dirty_price:.5f}\n"))

if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)